In [ ]:
import urllib.request
from bs4 import BeautifulSoup, NavigableString
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import selenium.common.exceptions as Exception

In [ ]:
BASE_PEOPLE = "https://letterboxd.com/people/popular/page/"
LBOX = "https://letterboxd.com"
REV = "films/reviews/page/"

userNames = list()

t0 = time.time()

# Get the usernames from people page by all-time popularity (set to: 1st page)
for i in range (1,2):
    URL = BASE_PEOPLE + str(i)
    soup = BeautifulSoup(urllib.request.urlopen(URL),"lxml")
    for tag in soup.find_all("h3", "title-3"):
        href = tag.a.get("href")
        userNames.append(href)
    
    time.sleep(0.25)
t1 = time.time()            
print(f"{t1-t0} seconds to download {len(userNames)} usernames.")

In [ ]:
len(userNames)

In [ ]:
# UNCOMMENT to install: !pip install webdriver-manager
from webdriver_manager.chrome import ChromeDriverManager
driver = webdriver.Chrome(ChromeDriverManager().install())  #using Selenium-Chrome simulator to interact with JS

In [ ]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [ ]:
# Store num. of pages of reviews to usernames
numPageReviews = dict()
# Store all data of reviews to usernames
allReviewData = dict()

driver = webdriver.Chrome(ChromeDriverManager().install()) 
t0 = time.time()

for name in userNames:
    # Find num. of pages of reviews from the first page
    firstPage = LBOX + name + REV + "1"
    
    soup = BeautifulSoup(urllib.request.urlopen(firstPage),"lxml")
    time.sleep(2)
    
    # Store user's all review data (movie/film name + year + review + rating)
    data = list()

    for tag in soup.find_all("div", "paginate-pages"):
        item = tag.find_all('li','paginate-page')
        numPageReviews[name] = (int(item[-1].text))
    
    # Initialize lists for data
    ratings = list()
    movies = list()
    reviews = list()
    
    # Loop through all pages and obtain data from each page
    for pageNum in range(1, numPageReviews[name]+1):
        thePage = LBOX + name + REV + str(pageNum)
        driver.maximize_window()
        driver.get(thePage)
        time.sleep(10)
        # Click on the spoilers links
        spoilers = list()
        spoilers = driver.find_elements_by_link_text('I can handle the truth.')
        state = True
        if len(spoilers)==0:
            state = False   
        if state ==True:
            for i in range(len(spoilers)):
                time.sleep(7)
                attempts = 0
                while(attempts < 2):
                    try:
                        WebDriverWait(driver,20).until(EC.presence_of_element_located((By.LINK_TEXT,'I can handle the truth.'))).click()
                        break
                    except Exception as exc:
                        print("Error " + exc + " at :" + thePage + " at entry: " + str(i))
                    attempts += 1
        # Click on the "more" links to reveal all text            
        linksMore = list()
        time.sleep(7)
        linksMore = driver.find_elements_by_class_name('reveal')
        state = True
        if len(linksMore)==0:
            state = False   
        if state == True:
            for i in range(len(linksMore)):
                time.sleep(5)
                attempts = 0
                while(attempts < 2):
                    try:
                        WebDriverWait(driver,20).until(EC.presence_of_element_located((By.CLASS_NAME,'reveal'))).click()
                        break
                    except Exception as exc:
                        print("Error " + exc + " at :" + thePage + " at entry: " + str(i))
                    attempts += 1
        
        time.sleep(7)
        soup = BeautifulSoup(driver.page_source,"html.parser")
        
        ratingList = list()  # list to store the review entries with ratings only      
        for tag in soup.find_all("p", "attribution"):
            spans = tag.find_all('span','rating')
            ratingList.append(spans)
            for span in spans:
                rating = [i for i in str(span.attrs.get('class')) if i in "0123456789"]
                if len(rating)==2:
                    rating = ['10']
                ratings.append(rating[0])

        for i, tag in enumerate(soup.find_all("h2", "headline-2 prettify")):
            if not ratingList[i]:
                continue
            filmName = tag.a
            filmYear = tag.small.a
            for film in zip(filmName, filmYear):
                movies.append(film)
        

        for i, tag in enumerate(soup.find_all("div", "body-text")):
            if not ratingList[i]:
                continue
            rev = ""
            for item in tag:
                if isinstance(item, NavigableString):
                    continue
                rev += str(item.text)
            reviews.append(cleanhtml(rev))
            
    for i in range(len(ratings)):
        data.append([movies[i],reviews[i],ratings[i]])
        
    allReviewData[name] = data
    
t1 = time.time()

In [ ]:
print(f"{(t1-t0)/60} minutes to download reviews of {len(allReviewData)} usernames.")